## TO DO

3. check reference chromosome

4. pull data from gene and variant databases (prefereably already hosted online)

In [1]:
using JuliaDB
using IndexedTables
using Dagger
using VCFTool

# For viewing DataFrames

ENV["COLUMNS"]=100

ENV["LINES"]=200

# For viewing JuliaDB tables and arrays

IndexedTables.set_show_compact!(false);

┌ Info: Precompiling VCFTool [8f8d80d9-e796-45a3-8ef2-c0461a3110a4]
└ @ Base loading.jl:1278


In [2]:
input_dir = "../input/"

output_dir = "../output/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_738_1000_file_path = joinpath(input_dir, "738_variants_1000.vcf.gz")

vcf_738_1000_dagger_file_path = joinpath(input_dir, "738_variants_1000.vcf.dagger")

vcf_769_3_file_path = joinpath(input_dir, "769_3_variants.vcf.gz")

vcf_test_file_path = joinpath(input_dir, "test.vcf.gz")

vcf_file_path_to_use = vcf_738_1000_file_path;

In [ ]:
# https://www.snpedia.com/index.php/Rs1042522
rs1042522 = ["chr17", 7676154]

# TP53
tp53 = ["chr17", 7661779:7687550]

# test set
t = ["chr1", 10000:12000];

## Tabix Regions

In [ ]:
# bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

# tabix_regions_from_file(
#     bed_file_path,
#     vcf_738_file_path,
#     output_dir,
# )

In [ ]:
# include("print_and_run_cmd.jl")

function search_region_with_tabix(vcf_gz_file_path, chromosome, start_position, end_position)
    
    run(pipeline(`tabix $vcf_gz_file_path $chromosome:$start_position-$end_position`))
    
end

In [ ]:
search_region_with_tabix(vcf_file_path_to_use, "chr1", 1200, 12000)

## Develop faidx

In [ ]:
benchmark_chrom = "chr1"

benchmark_chrom_int = 1

benchmark_variant = 13868

benchmark_region_start = 10000

benchmark_region_end = 900000;

benchmark_variant_2 = 19322

benchmark_region_2_start = 500000

benchmark_region_2_end = 800000;

In [ ]:
;samtools faidx ../input/reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna chr1:12000-12200

## Search for region of VCF using Julia DB

In [ ]:
vcf_table = make_vcf_indexedtable(vcf_file_path_to_use);

In [ ]:
search_region(vcf_738_1000_dagger_file_path, "chr1", 13867, 13869)

In [ ]:
search_region(vcf_738_1000_dagger_file_path, "chr1", 12000, 16000)